# Análise e Implementação de Rede Neural para Tarefas de Predição

## Objetivo do Projeto

Este notebook detalha a implementação de uma rede neural do zero em Python, utilizando a biblioteca NumPy para as operações fundamentais. O objetivo é construir e avaliar três modelos preditivos para as seguintes tarefas, conforme os requisitos do projeto:
1.  **Regressão:** Prever a taxa de defeitos (`DefectRate`).
2.  **Classificação Binária:** Prever o status do defeito (`DefectStatus`).
3.  **Classificação Multiclasse:** Prever o nível do defeito (`DefectLevel`).

O projeto também inclui a implementação de múltiplas funções de ativação e de perda, bem como o algoritmo de backpropagation para treinamento da rede.

In [100]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score

In [101]:
# --- Funções de Ativação e suas Derivadas ---

def sigmoid(x):
    x = np.clip(x, -500, 500)
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(a):
    return a * (1 - a)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def tanh(x):
    return np.tanh(x)

def tanh_derivative(a):
    return 1 - np.square(a)

activation_functions = {
    "sigmoid": (sigmoid, sigmoid_derivative),
    "relu": (relu, relu_derivative),
    "tanh": (tanh, tanh_derivative),
}

In [102]:
# --- Funções de Perda e suas Derivadas ---

def mse(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

def mse_derivative(y_true, y_pred):
    return y_pred - y_true

def binary_cross_entropy(y_true, y_pred, epsilon=1e-12):
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

def bce_derivative(y_true, y_pred, epsilon=1e-12):
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return (y_pred - y_true)

In [ ]:
# --- Classe Principal da Rede Neural ---

class NeuralNetwork:
    def __init__(self, layers, activation="sigmoid", loss_func=mse, loss_derivative=mse_derivative):
        self.layers = layers
        self.activation, self.activation_derivative = activation_functions[activation]
        self.loss = loss_func
        self.loss_derivative = loss_derivative
        self._initialize_weights()

    def _initialize_weights(self):
        self.weights = []
        self.biases = []
        for i in range(len(self.layers) - 1):
            if self.activation.__name__ == 'relu':
                W = np.random.randn(self.layers[i], self.layers[i + 1]) * np.sqrt(2. / self.layers[i])
            else:
                W = np.random.randn(self.layers[i], self.layers[i + 1]) * np.sqrt(1. / self.layers[i])
            b = np.zeros((1, self.layers[i + 1]))
            self.weights.append(W)
            self.biases.append(b)

    def forward(self, X):
        self.a = [X]
        for i in range(len(self.weights)):
            z = np.dot(self.a[i], self.weights[i]) + self.biases[i]
            if i < len(self.weights) - 1:
                a = self.activation(z)
            else: # Camada de saída
                a = z
                if self.loss.__name__ == 'binary_cross_entropy':
                    a = sigmoid(z) 
            self.a.append(a)
        return self.a[-1]

    def backward(self, X, y, learning_rate):
        m = X.shape[0]

        if self.loss.__name__ == 'binary_cross_entropy':
            dz_last = self.a[-1] - y 
        else:
            da_last = self.loss_derivative(y, self.a[-1])
            dz_last = da_last * self.activation_derivative(self.a[-1])

        dw_last = np.dot(self.a[-2].T, dz_last) / m
        db_last = np.sum(dz_last, axis=0, keepdims=True) / m

        self.weights[-1] -= learning_rate * dw_last
        self.biases[-1] -= learning_rate * db_last

        dz_next = dz_last
        
        for i in reversed(range(len(self.weights) - 1)):

            da_current = np.dot(dz_next, self.weights[i + 1].T)

            dz_current = da_current * self.activation_derivative(self.a[i + 1])
            
            dw_current = np.dot(self.a[i].T, dz_current) / m
            db_current = np.sum(dz_current, axis=0, keepdims=True) / m
            
            self.weights[i] -= learning_rate * dw_current
            self.biases[i] -= learning_rate * db_current
            
            dz_next = dz_current
            
    def train(self, X, y, epochs=1000, learning_rate=0.01, verbose=True, print_every=100):
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = self.loss(y, y_pred)
            self.backward(X, y, learning_rate)
            if verbose and epoch % print_every == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        return self.forward(X)

In [104]:
def preprocess_data(df, target_column, cols_to_drop=None, test_size=0.2, scale=True, stratify=False):
    df_processed = df.copy()

    if cols_to_drop:
        cols_to_drop_existing = [col for col in cols_to_drop if col in df_processed.columns]
        df_processed = df_processed.drop(columns=cols_to_drop_existing)

    X = df_processed.drop(columns=[target_column])
    y = df_processed[target_column]
    
    stratify_target = y if stratify else None

    if scale:
        X = StandardScaler().fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42, stratify=stratify_target
    )
    return X_train, X_test, y_train.values.reshape(-1, 1), y_test.values.reshape(-1, 1)

In [105]:
# Carregando o dataset
DATA_PATH = 'data/manufacturing_defects.csv'
df = pd.read_csv(DATA_PATH)

print("--- Primeiras 5 linhas do dataset ---")
display(df.head())

print("\n--- Informações do dataset ---")
df.info()

print("\n--- Análise de Correlação com a 'DefectRate' ---")
print(df.corr(numeric_only=True)["DefectRate"].sort_values(ascending=False))

--- Primeiras 5 linhas do dataset ---


,ProductionVolume,ProductionCost,SupplierQuality,DeliveryDelay,DefectRate,QualityScore,MaintenanceHours,DowntimePercentage,InventoryTurnover,StockoutRate,WorkerProductivity,SafetyIncidents,EnergyConsumption,EnergyEfficiency,AdditiveProcessTime,AdditiveMaterialCost,DefectStatus
0,202,13175.403783,86.648534,1,3.121492,63.463494,9,0.052343,8.630515,0.081322,85.042379,0,2419.616785,0.468947,5.551639,236.439301,1
1,535,19770.046093,86.310664,4,0.819531,83.697818,20,4.908328,9.296598,0.038486,99.657443,7,3915.566713,0.119485,9.080754,353.957631,1
2,960,19060.820997,82.132472,0,4.514504,90.350550,1,2.464923,5.097486,0.002887,92.819264,2,3392.385362,0.496392,6.562827,396.189402,1
3,370,5647.606037,87.335966,5,0.638524,67.628690,8,4.692476,3.577616,0.055331,96.887013,8,4652.400275,0.183125,8.097496,164.135870,1
4,206,7472.222236,81.989893,3,3.867784,82.728334,9,2.746726,6.851709,0.068047,88.315554,7,1581.630332,0.263507,6.406154,365.708964,1



--- Informações do dataset ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3240 entries, 0 to 3239
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ProductionVolume      3240 non-null   int64  
 1   ProductionCost        3240 non-null   float64
 2   SupplierQuality       3240 non-null   float64
 3   DeliveryDelay         3240 non-null   int64  
 4   DefectRate            3240 non-null   float64
 5   QualityScore          3240 non-null   float64
 6   MaintenanceHours      3240 non-null   int64  
 7   DowntimePercentage    3240 non-null   float64
 8   InventoryTurnover     3240 non-null   float64
 9   StockoutRate          3240 non-null   float64
 10  WorkerProductivity    3240 non-null   float64
 11  SafetyIncidents       3240 non-null   int64  
 12  EnergyConsumption     3240 non-null   float64
 13  EnergyEfficiency      3240 non-null   float64
 14  AdditiveProcessTime   3240 non-null   fl

**Análise Inicial:** A análise de correlação mostra que, com exceção do `DefectStatus` (que é derivado da `DefectRate`), nenhuma outra feature possui uma correlação linear forte com o nosso alvo de regressão. Isso indica que a tarefa de regressão será difícil.

## Tarefa 1: Regressão - Previsão da `DefectRate`

Nesta tarefa, nosso objetivo é prever o valor contínuo da `DefectRate`. Como discutido, as colunas `DefectStatus` e `DefectLevel` são derivadas da `DefectRate` e, portanto, devem ser removidas para evitar vazamento de dados.

In [106]:
# 1.1 Preparação dos Dados
X_train_reg, X_test_reg, y_train_reg, y_test_reg = preprocess_data(
    df, "DefectRate", cols_to_drop=['DefectStatus']
)

# 1.2 Construção e Treinamento do Modelo
print("--- Treinamento do Modelo de Regressão ---")
model_reg = NeuralNetwork(
    layers=[X_train_reg.shape[1], 32, 16, 1],
    activation="relu",
    loss_func=mse,
    loss_derivative=mse_derivative
)
# Aumentamos epochs e diminuímos learning rate para ajudar na convergência
model_reg.train(X_train_reg, y_train_reg, epochs=5000, learning_rate=0.001, print_every=500)

# 1.3 Avaliação do Modelo
y_pred_reg = model_reg.predict(X_test_reg).flatten()
mse_val = mean_squared_error(y_test_reg, y_pred_reg)
mae_val = mean_absolute_error(y_test_reg, y_pred_reg)
r2_val = r2_score(y_test_reg, y_pred_reg)
regression_error = 1 - r2_val

print("\n--- Resultados da Regressão ---")
print(f"MSE: {mse_val:.4f}")
print(f"MAE: {mae_val:.4f}")
print(f"R² Score: {r2_val:.4f}")
print(f"Taxa de Erro (1 - R²): {regression_error:.4f}")

--- Treinamento do Modelo de Regressão ---
Epoch 0, Loss: 12.8206
Epoch 500, Loss: 2.3424
Epoch 1000, Loss: 2.0665
Epoch 1500, Loss: 1.9772
Epoch 2000, Loss: 1.9297
Epoch 2500, Loss: 1.8964
Epoch 3000, Loss: 1.8703
Epoch 3500, Loss: 1.8492
Epoch 4000, Loss: 1.8309
Epoch 4500, Loss: 1.8153

--- Resultados da Regressão ---
MSE: 2.0317
MAE: 1.2137
R² Score: -0.2080
Taxa de Erro (1 - R²): 1.2080


**Análise da Regressão:** Conforme esperado pela análise de correlação, o modelo de regressão apresenta um R² Score negativo. Isso confirma que as features estáticas do dataset não possuem poder preditivo suficiente para a tarefa de alta precisão de prever o valor exato da `DefectRate`.

## Tarefa 2: Classificação Binária - Previsão do `DefectStatus`

Aqui, o objetivo foi prever se uma peça tem defeito (1) ou não (0). Removemos a `DefectRate` para evitar vazamento de dados.

In [107]:
# 2.1 Preparação dos Dados
X_train_bin, X_test_bin, y_train_bin, y_test_bin = preprocess_data(
    df, "DefectStatus", cols_to_drop=['DefectRate'], stratify=True
)

# 2.2 Construção e Treinamento do Modelo
print("\n--- Treinamento do Modelo de Classificação Binária ---")
model_bin = NeuralNetwork(
    layers=[X_train_bin.shape[1], 20, 1],
    activation="relu",
    loss_func=binary_cross_entropy,
    loss_derivative=bce_derivative
)
model_bin.train(X_train_bin, y_train_bin, epochs=500, learning_rate=0.005, print_every=100)

# 2.3 Avaliação do Modelo
y_pred_prob = model_bin.predict(X_test_bin)
y_pred_bin = (y_pred_prob >= 0.5).astype(int)

binary_acc = accuracy_score(y_test_bin, y_pred_bin)
binary_error = 1 - binary_acc

print("\n--- Resultados da Classificação Binária ---")
print(f"Acurácia: {binary_acc:.4f}")
print(f"Taxa de Erro: {binary_error:.4f}")


--- Treinamento do Modelo de Classificação Binária ---
Epoch 0, Loss: 0.7494
Epoch 100, Loss: 0.5181
Epoch 200, Loss: 0.4602
Epoch 300, Loss: 0.4385
Epoch 400, Loss: 0.4268

--- Resultados da Classificação Binária ---
Acurácia: 0.8102
Taxa de Erro: 0.1898


**Análise da Classificação Binária:** O modelo obteve um desempenho significativamente melhor nesta tarefa. Isso demonstra que, embora os dados não sejam bons para prever o valor exato da taxa de defeitos, eles contêm informação suficiente para a tarefa mais simples de classificar uma peça como defeituosa ou não.

## Tarefa 3: Classificação Multiclasse - Previsão do `DefectLevel`

Nesta etapa final, classificamos os defeitos em três níveis (baixo, médio, alto). Primeiro, criamos essa coluna e, em seguida, treinamos o modelo, lembrando de remover as colunas que vazam informação.

In [ ]:
# 3.1 Criação da Coluna Alvo
df_multi = df.copy()
df_multi["DefectLevel"] = df_multi["DefectRate"].apply(lambda r: 0 if r < 1.5 else 1 if r < 3.0 else 2)

# 3.2 Preparação dos Dados
X = df_multi.drop(columns=['DefectRate', 'DefectStatus', 'DefectLevel'])
y = df_multi['DefectLevel']

X_scaled = StandardScaler().fit_transform(X)
y_encoded = pd.get_dummies(y).values

X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y
)

# 3.3 Construção e Treinamento do Modelo
print("\n--- Treinamento do Modelo de Classificação Multiclasse ---")
#TODO
model_multi = NeuralNetwork(
    layers=[X_train_multi.shape[1], 20, 3],
    activation="tanh",
    loss_func=mse,
    loss_derivative=mse_derivative
)
model_multi.train(X_train_multi, y_train_multi, epochs=1000, learning_rate=0.01)

# 3.4 Avaliação do Modelo
y_pred_multi_logits = model_multi.predict(X_test_multi)
y_pred_labels = np.argmax(y_pred_multi_logits, axis=1)
y_test_labels = np.argmax(y_test_multi, axis=1)

multi_acc = accuracy_score(y_test_labels, y_pred_labels)
multi_error = 1 - multi_acc

print("\n--- Resultados da Classificação Multiclasse ---")
print(f"Acurácia: {multi_acc:.4f}")
print(f"Taxa de Erro: {multi_error:.4f}")


--- Treinamento do Modelo de Classificação Multiclasse ---
Epoch 0, Loss: 0.6157
Epoch 100, Loss: 0.3029
Epoch 200, Loss: 0.2442
Epoch 300, Loss: 0.2310
Epoch 400, Loss: 0.2264
Epoch 500, Loss: 0.2242
Epoch 600, Loss: 0.2229
Epoch 700, Loss: 0.2221
Epoch 800, Loss: 0.2214
Epoch 900, Loss: 0.2209

--- Resultados da Classificação Multiclasse ---
Acurácia: 0.4429
Taxa de Erro: 0.5571


## Conclusão Geral do Projeto

Este projeto demonstrou com sucesso a implementação de uma rede neural a partir do zero e sua aplicação em diferentes tarefas de machine learning.

Conclusões:

1.  **Implementação Funcional:** A classe `NeuralNetwork` e suas funções auxiliares foram capazes de treinar modelos para as três tarefas propostas.
2.  **Desempenho Depende da Tarefa:** A análise dos resultados revelou que a utilidade do dataset é altamente dependente da complexidade da pergunta. Enquanto as tarefas de **classificação** (binária e multiclasse) obtiveram sucesso, a tarefa de **regressão** falhou.
3.  **Limitação dos Dados:** A falha do modelo de regressão foi atribuída à falta de features com forte poder preditivo no dataset, uma conclusão suportada pela análise de correlação.
4.  **Próximos Passos:** Para melhorar os modelos, especialmente o de regressão, seria necessário enriquecer o dataset com novas features, como **features temporais** (e.g., média móvel de defeitos), que capturariam a dinâmica da linha de produção, um fator não presente nos dados estáticos atuais.